In [1]:
import time
import datetime

import cst_python as cst
import toolpy as tp
from toolpy.integrations import groq

import gbyg as gg

In [2]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [3]:
groq_interface = groq.GroqInterface(model=groq.GroqModel.LLAMA3_70B, n_retry=5)

registry = tp.llm.LLMRegistry()
registry.registry(model_name="llama3-70b", interface=groq_interface, default=True)

In [4]:
importance_threshould = 1

In [5]:
mind = cst.Mind()

gg.agent.agent_constructor(mind, importance_threshould)

for codelet in mind.code_rack.all_codelets:
    codelet.time_step = 100

memories_input = mind.raw_memory.get_all_of_type("MemoriesInput")[0]
memory_stream = mind.raw_memory.get_all_of_type("MemoryStream")[0]
agent_time = mind.raw_memory.get_all_of_type("AgentTime")[0]
agent_info = mind.raw_memory.get_all_of_type("AgentInfo")[0]
agent_summary_description = mind.raw_memory.get_all_of_type("AgentSummaryDescription")[0]

generated_questions = mind.raw_memory.get_all_of_type("GeneratedQuestions")[0]
statements = mind.raw_memory.get_all_of_type("Statements")[0]

In [6]:
agent_info.set_info({"name":"Alex", "Age":25, "Traits":"friendly, outgoing, hospitable"})
agent_time.set_info(datetime.datetime.fromisoformat("2024-06-26T00:00:00").timestamp())

-1

In [7]:
mind.start()

In [8]:
input_memories  = [
  {
    "index": 0,
    "type": "event",
    "created": "2024-06-01 08:00:00",
    "description": "Alex attended a nature photography workshop in the city park.",
  },
  {
    "index": 1,
    "type": "reflection",
    "created": "2024-06-01 19:00:00",
    "description": "Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.",
  },
  {
    "index": 2,
    "type": "event",
    "created": "2024-06-05 10:00:00",
    "description": "Alex completed a major software development project at work, delivering the final product to the client.",
  },
  {
    "index": 3,
    "type": "reflection",
    "created": "2024-06-05 20:00:00",
    "description": "After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.",
  },
  {
    "index": 4,
    "type": "event",
    "created": "2024-06-10 16:00:00",
    "description": "Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.",
  }
]

for m in input_memories:
    m["created"] = datetime.datetime.fromisoformat(m["created"]).timestamp()
    m["last_acessed"] = m["created"]

In [9]:
input_memories

[{'index': 0,
  'type': 'event',
  'created': 1717239600.0,
  'description': 'Alex attended a nature photography workshop in the city park.',
  'last_acessed': 1717239600.0},
 {'index': 1,
  'type': 'thought',
  'created': 1717279200.0,
  'description': 'Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.',
  'last_acessed': 1717279200.0},
 {'index': 2,
  'type': 'event',
  'created': 1717592400.0,
  'description': 'Alex completed a major software development project at work, delivering the final product to the client.',
  'last_acessed': 1717592400.0},
 {'index': 3,
  'type': 'thought',
  'created': 1717628400.0,
  'description': 'After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
  'last_acessed': 1717628400.0},
 {'index': 4,
  'type': 'event',
  'created': 1718046000.0,
  'description': 'Alex went on a weekend trip to explore a national park, taking photos

In [10]:
memories_input_queue = memories_input.get_info()
for memory in input_memories:
    memories_input_queue.append(memory)

In [11]:
time.sleep(10)

print(len(memory_stream.get_info()))

D:\Github\GDataByGAgents\src\gbyg\agent\retrieval\recency_scorer_tool.py:32: RuntimeWarning: invalid value encountered in divide
  scores = (scores-min_score)/(max_score-min_score)
c:\Python38\lib\site-packages\sentence_transformers\util.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  b = torch.tensor(b)


10


In [12]:
questions = generated_questions.get_info()
for q in questions:
    print(q)

What kind of activities does Alex enjoy doing in his free time?
What are Alex's professional accomplishments?
What skills has Alex developed or improved?


In [13]:
for memory in memory_stream.get_info():
    because_of = ""
    if 'because_of' in memory: 
        because_of = memory['because_of']

    memory_str = f"{memory['index']} ({memory['type']}): {memory['description']} | because of: {because_of}"
    print(memory_str)

0 (event): Alex attended a nature photography workshop in the city park. | because of: 
1 (thought): Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects. | because of: 
2 (event): Alex completed a major software development project at work, delivering the final product to the client. | because of: 
3 (thought): After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome. | because of: 
4 (event): Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes. | because of: 
5 (reflection): Alex has a strong desire for self-improvement and learning | because of: [0, 1]
6 (reflection): Alex has experienced a significant level of accomplishment and relief | because of: [2, 3]
7 (reflection): Alex enjoys exploring and appreciating nature | because of: [0, 4]
8 (reflection): Alex has multiple interests and hobbies | because of: [0, 2, 4]
9

In [14]:
print(agent_summary_description.get_info())

{'name': 'Alex', 'Age': 25, 'Traits': 'friendly, outgoing, hospitable'}
Alex is likely an outdoor enthusiast and an art lover.
Alex is likely a photographer or has an interest in photography.
Alex seems to be taking steps to explore new interests and hobbies, possibly indicating a sense of excitement and enthusiasm about personal growth and self-improvement.


In [16]:
agent_time.set_info(agent_time.get_info()+gg.time_utils.TimeInterval.ONE_DAY)
time.sleep(5)
print(agent_summary_description.get_info())

{'name': 'Alex', 'Age': 25, 'Traits': 'friendly, outgoing, hospitable'}
Alex is an appreciative and accomplished individual with a love for nature and a sense of relief from past achievements.
Alex is a task-oriented person who takes breaks to relax and unwind after achieving significant accomplishments.
Alex is feeling fulfilled and rejuvenated


In [17]:
print(len(memory_stream.get_info()))

10
